# Policy Gradient for Optimal Control

In this notebook, we will empirically test policies for various environments, to measure the efficacy of the policies we develop in the written section on live noise. We will be using the environments defined in `environments/multiplicative_gaussian_noise_environment.py`, which will need to be completed prior to beginning this notebook. There are certain properties about these environments that are more specific than definition 2 of section 1.2 of the writeup. Here is an overview of those:
1. $A_t \sim \mathcal{N}(a, \alpha^2)$, $B_t \sim \mathcal{N}(b, \beta^2)$, and $C_t \sim \mathcal{N}(c, \gamma^2)$ are all normal distributions.
2. $X_0 = 1$ (this means $\mu = 1$, $\sigma = 0$).

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from tqdm import tqdm
from typing import List
from torch import Tensor
from environments.base_environment import Environment
from environments.multiplicative_gaussian_noise_environment import MultiplicativeGaussianNoiseEnvironment, MultiplicativeGaussianControlNoiseEnvironment, \
                                                    MultiplicativeGaussianStateControlNoiseEnvironment, MultiplicativeGaussianObservationNoiseEnvironment

from policies.base_policy import Policy
from policies.m1p1_linear_policy_modules import FixedWeightM1P1LinearModule

from infrastructure.visualization import plot_average_second_moment_trajectory, plot_empirical_average_second_moment_and_partial_loss, plot_empirical_loss_over_a
from agent import Agent
from control_engine import ControlEngine

import random
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

In [ ]:
import os

# Patch for buggy matplotlib
os.environ['KMP_DUPLICATE_LIB_OK']='True'

The `sim_policy` function simulates $N$ trajectories, each of $T$ timesteps, using the given policy within the given environment.

In [ ]:
def sim_policy(T: int, N: int, policy: Policy, env: Environment):
    agent = Agent(policy)
    engine = ControlEngine(agent, env)
    states, _, _, losses = engine.collect_trajectories(N = 100, T = 100, use_stochastic_policy = False)
    return states, losses

# Control Noise

Write the optimal linear coefficient $\theta$ you found in Problem 3 for the control noise system as a function of $a$, $b$, and $\beta$.

In [ ]:
# TODO
theta_cn = lambda a, b, beta: ...

Analyze the system where $a = 1$, $b = 1$, $\beta = 1$.

In [ ]:
a = 1
b = 1
beta = 1

env = MultiplicativeGaussianControlNoiseEnvironment(
    a = a,
    b = b,
    beta = beta,
    lmbda = 0
)

linear_policy = Policy(FixedWeightM1P1LinearModule(theta_cn(a, b, beta)))
states, losses = sim_policy(100, 100, linear_policy, env)

We empirically estimate $\mathbb E[X_t^2] \approx \frac{1}{N} \sum_{i=1}^{N} (X^{(i)})^2_t$, where $X^{(i)}_t$ is the state at timestep $t$ on the $i$-th rollout.

Likewise, we define the empirical loss as $\overline{L}(F) = \sum_{t=1}^{T} \mathbb E[X_t^2] \approx \frac{1}{N} \sum_{t=1}^{T} \sum_{i=1}^{N} \Big(X^{(i)}_t\Big)^2$. It is sometimes interesting to look at the 'partial' loss; that is, the loss for the first $\tilde{t}$ timesteps. Define this as $\overline{L}(F; \tilde{t}) = \frac{1}{N} \sum_{t=1}^{\tilde{t}} \sum_{i=1}^{N} \Big(X^{(i)}_t\Big)^2$.

In [ ]:
plot_empirical_average_second_moment_and_partial_loss(states, losses, "Control-Noise")

Now, we rerun the above simulation with different values of $a$ (again with $b = 1$ and $\beta = 1$).

In [ ]:
a_list = [3, 2, 1.5, 1.4, 1.0, 0.9]

a_losses = []

for a in a_list:
    env = MultiplicativeGaussianControlNoiseEnvironment(
        a = a,
        b = b,
        beta = beta,
        lmbda = 0
    )
    
    linear_policy = Policy(FixedWeightM1P1LinearModule(theta_cn(a, b, beta)))
    states, losses = sim_policy(100, 100, linear_policy, env)
    plot_average_second_moment_trajectory(states, a)

### What values of $a$ are stabilizable by your analysis? Do the above plots line up with your analysis?

TODO

Now, we will examine an interesting property of the optimal policy derived in Problem 3 (b). Let $F^*$ denote this policy, and let $U_{t}^{*} = F_{t}^{*}(Y_{(t)})$. It's worth noting that this policy does not necessarily set the expected value of $X_{t + 1}$ to $0$; that is,

$$\mathbb E[X_{t+1} | Y_t] = \mathbb E[aY_t + B_tU_t^* \mid Y_{t}] \neq 0$$

whenever $\beta > 0$. We say that this policy is "biased".

We could compute another ("unbiased") policy $F^{0}$ such that 
$$\mathbb E[aY_t + B_tU_t^0 \mid Y_{t}] = 0.$$
This policy is given by $U_t^0 = F_t^0(Y_{(t)}) = -\frac{a}{b}Y_t$. When $\beta \gg 0$, the difference between these policies is quite pronounced. Since this policy does not optimize the second moment, it will perform worse (in the second-moment sense) when you compare it to the optimal second-moment-minimizing policy.

In [ ]:
# TODO
theta_0 = lambda a, b, beta: ...

Compare both policies when $a = 1.1$, $b = 1$, and $\beta = 2$.

In [ ]:
a = 1.1
b = 1
beta = 2

env = MultiplicativeGaussianControlNoiseEnvironment(
    a = a,
    b = b,
    beta = beta,
    lmbda = 0
)

### U^*
linear_policy = Policy(FixedWeightM1P1LinearModule(theta_cn(a, b, beta)))
states_star, losses_star = sim_policy(100, 100, linear_policy, env)

### U^0
linear_policy = Policy(FixedWeightM1P1LinearModule(theta_0(a, b, beta)))
states_0, losses_0 = sim_policy(100, 100, linear_policy, env)

In [ ]:
mean_states_star = np.array(states_star).mean(axis=0)
mean_second_moments_star = np.power(states_star, 2).mean(axis=0)
mean_cumsum_losses_star = np.array(losses_star).cumsum(axis=1).mean(0)

mean_states_0 = np.array(states_0).mean(axis=0)
mean_second_moments_0 = np.power(states_0, 2).mean(axis=0)
mean_cumsum_losses_0 = np.array(losses_0).cumsum(axis=1).mean(0)

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(9,18))

ax[0].plot(mean_second_moments_star, label='biased')
ax[0].plot(mean_second_moments_0, label='unbiased')
ax[0].set_yscale("log")
ax[0].set_xlabel(r'$t$')
ax[0].set_ylabel(r'$\overline{X_t^2}$')
ax[0].legend()
ax[0].set_title('Empirical $\mathbb{E}[X_{t}^{2}]$ for Control-Noise System')

ax[1].plot(mean_cumsum_losses_star, label='biased')
ax[1].plot(mean_cumsum_losses_0, label='unbiased')
ax[1].set_xlabel(r'$\tilde{t}$')
ax[1].set_ylabel(r'$L(F; \tilde{t})$')
ax[1].legend()
ax[1].set_title('Empirical Partial Loss for Control-Noise System')
plt.show()

### What do you notice about the difference in the empirical average state second moment $\mathbb{E}[X_t^2]$ of the biased versus unbiased policy? Which one is more stable?

TODO

# State-Control Noise

Write the optimal linear coefficient $\theta$ you found in Problem 4 for the state-control noise system as a function of $a$, $b$, $\alpha$, $\beta$, and $\lambda$.

In [ ]:
# TODO
theta_scn = lambda a, b, alpha, beta, lmbda: ...

Analyze the system where $a = 1$, $b = 1$, $\alpha = 0.5$, $\beta = 0.5$, $\lambda = 1$.

In [ ]:
a = 1
b = 1
alpha = 0.5
beta = 0.5
lmbda = 1

env = MultiplicativeGaussianStateControlNoiseEnvironment(
    a = a,
    b = b,
    alpha = alpha,
    beta = beta,
    lmbda = lmbda
)

linear_policy = Policy(FixedWeightM1P1LinearModule(theta_scn(a, b, alpha, beta, lmbda)))
states, losses = sim_policy(100, 100, linear_policy, env)

Plot the empirical average second moment and the partial empirical losses as we did for the control noise system.

In [ ]:
plot_empirical_average_second_moment_and_partial_loss(states, losses, "State-Control-Noise")

As we did in the case of Control Noise, for a variety of $a$ (and with $b = 1$, $\alpha = 0.5$, $\beta = 0.5$, and $\lambda = 1$ as above), we rerun the above simulation

In [ ]:
a_list = [3.0, 2.0, 1.4, 1.3, 1.0, 0.9]

a_losses = []

for a in a_list:
    env = MultiplicativeGaussianStateControlNoiseEnvironment(
        a = a,
        b = b,
        alpha = alpha,
        beta = beta,
        lmbda = lmbda
    )
    
    linear_policy = Policy(FixedWeightM1P1LinearModule(theta_scn(a, b, alpha, beta, lmbda)))
    states, losses = sim_policy(100, 100, linear_policy, env)
    plot_average_second_moment_trajectory(states, a)

### What values of $a$ can be stabilized by your analysis?

TODO

# Observation Noise 

Write the optimal linear coefficient $\theta$ you found in Problem 5 for the observation noise system as a function of $a$, $c$, and $\gamma$.

In [ ]:
# TODO
theta_on = lambda a, c, gamma: ...

Analyze the system where $a = 1$, $c = 1$, and $\gamma = 1$.

In [ ]:
a = 1
c = 1
gamma = 1

env = MultiplicativeGaussianObservationNoiseEnvironment(
    a = a,
    c = c,
    gamma = gamma,
    lmbda = 0
)

linear_policy = Policy(FixedWeightM1P1LinearModule(theta_on(a, c, gamma)))
states, losses = sim_policy(100, 100, linear_policy, env)

Plot the empirical average second moment and the partial empirical losses.

In [ ]:
plot_empirical_average_second_moment_and_partial_loss(states, losses, "Observation-Noise")

### How does the perfomance of this policy compare to the perfromance of the greedy linear policy in the control noise case illustrated in the begining of the notebook?

TODO

As we did in the case of Control and State-Control Noise systems, for a variety of $a$, $c = 1$, $\gamma = 1$ (as above), rerun the above simulation and save $\overline{L}(F^*)$.

In [ ]:
a_list = [3, 2, 1.5, 1.4, 1.0, 0.9]

a_losses = []

for a in a_list:
    env = MultiplicativeGaussianObservationNoiseEnvironment(
        a = a,
        c = c,
        gamma = gamma,
        lmbda = 0
    )

    linear_policy = Policy(FixedWeightM1P1LinearModule(theta_on(a, c, gamma)))
    states, losses = sim_policy(100, 100, linear_policy, env)
    plot_average_second_moment_trajectory(states, a)

### What values of $a$ are stabilizable by your analysis? Does the above graph line up with your analysis?

TODO

# Period-1 vs Period-2 Policies for Observation Noise

Implement the memory-1 period-2 policy defined in Problem 5 (f) as $F'_t(Y_{(t)}) = \begin{cases}\frac{1}{2} + \frac{2}{5}|Y_{t}| & \text{if \(t\) is even} \\ -\frac{1}{2} - \frac{1}{2}|Y_{t}| & \text{if \(t\) is odd.}\end{cases}$

In [ ]:
class FPrimeModule(torch.nn.Module):
    def forward(self, y_history: List[Tensor]):  # List[()] -> ()
        """
        Given the most recent observation, returns a control F_t(Y_t) = 0.5 + 0.4 * |Y_t| when t is even 
        and -0.5 - 0.5 * |Y_t| when t is odd.
        """
        t = len(y_history)

        # TODO

When $a = 1$, $c = 0$, $\gamma = 1$, compare $F^*$ (the policy derived in Problem 5 (b)) to $F'$. 

In [ ]:
a = 1
c = 0
gamma = 1

env = MultiplicativeGaussianObservationNoiseEnvironment(
    a = a,
    c = c,
    gamma = gamma,
    lmbda = 0
)

### F^*
linear_policy = Policy(FixedWeightM1P1LinearModule(theta_on(a, c, gamma)))
states_star, losses_star = sim_policy(100, 100, linear_policy, env)

### F^'
p2_policy = Policy(FPrimeModule())
states_prime, losses_prime = sim_policy(100, 100, p2_policy, env)

In [ ]:
mean_second_moments_star = np.power(states_star, 2).mean(axis=0)
mean_cumsum_losses_star = np.array(losses_star).cumsum(axis=1).mean(0)

mean_second_moments_prime = np.power(states_prime, 2).mean(axis=0)
mean_cumsum_losses_prime = np.array(losses_prime).cumsum(axis=1).mean(0)

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(9,12))

ax[0].plot(mean_second_moments_star, label=r"$F^*$")
ax[0].plot(mean_second_moments_prime, label=r"$F'$")
ax[0].set_xlabel(r'$t$')
ax[0].set_ylabel(r'$\mathbb{E}[X_t^2]$')
ax[0].set_yscale('log')
ax[0].legend()
ax[0].set_title('Empirical Average Second Moment for Observation-Noise System')

ax[1].plot(mean_cumsum_losses_star, label=r"$F^*$")
ax[1].plot(mean_cumsum_losses_prime, label=r"$F'$")
ax[1].set_xlabel(r'$\tilde{t}$')
ax[1].set_ylabel(r'$L(U; \tilde{t})$')
ax[1].legend()
ax[1].set_title('Partial Empirical Loss for Observation-Noise System')
plt.show()

### By comparing the performance of $F^*$ to the perfromance of $F'$, is $F^*$ the overall optimal policy for the observation noise system (as it is the case for the control noise and state-control noise systems)? 

TODO

Note from the plots above that the optimal linear memory-1 period-1 policy is in fact $F^*(Y_t) = 0$ when $C = 0$. Hence, $X_{t} = a^t\cdot X_0$ so when $|a| > 1$, this policy will necessarily not stabilize in the second moment. It remains unanswered, however, whether $F'$ can stabilize a system with $|a| > 1$.

Repeat the above analysis with $a = 1.01$, $c = 0$, $\gamma = 1$.

In [ ]:
a = 1.01
c = 0
gamma = 1

env = MultiplicativeGaussianObservationNoiseEnvironment(
    a = a,
    c = c,
    gamma = gamma,
    lmbda = 0
)

### F^*
linear_policy = Policy(FixedWeightM1P1LinearModule(theta_on(a, c, gamma)))
states_star, losses_star = sim_policy(100, 100, linear_policy, env)

### F^'
p2_policy = Policy(FPrimeModule())
states_prime, losses_prime = sim_policy(100, 100, p2_policy, env)

In [ ]:
mean_second_moments_star = np.power(states_star, 2).mean(axis=0)
mean_cumsum_losses_star = np.array(losses_star).cumsum(axis=1).mean(0)

mean_second_moments_prime = np.power(states_prime, 2).mean(axis=0)
mean_cumsum_losses_prime = np.array(losses_prime).cumsum(axis=1).mean(0)

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(9,12))

ax[0].plot(mean_second_moments_star, label=r"$F^*$")
ax[0].plot(mean_second_moments_prime, label=r"$F'$")
ax[0].set_xlabel(r'$t$')
ax[0].set_ylabel(r'$\mathbb{E}[X_t^2]$')
ax[0].set_yscale('log')
ax[0].legend()
ax[0].set_title('Empirical $\mathbb{E}[X_t^2]$ for Observation-Noise System')

ax[1].plot(mean_cumsum_losses_star, label=r"$F^*$")
ax[1].plot(mean_cumsum_losses_prime, label=r"$F'$")
ax[1].set_xlabel(r'$\tilde{t}$')
ax[1].set_ylabel(r'$L(F); \tilde{t})$')
ax[1].legend()
ax[1].set_title('Empirical Partial Loss for Observation-Noise System')
plt.show()

### By inspection, does $F'$ seem to stabilize this system?

TODO

# Playground - Try something cool!